# CS 447 Homework 1 $-$ Language Models & Morphological Transduction
In this homework we will study some traditional appraoches to a few natural language tasks. First, you will build some n-gram language models on a corpus of Wikipedia articles, and then you will design a finite-state transducer for verb conjugation in English.


#Part 1: Language Models [60 points]

Here, you will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.


## Preprocessing the Data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. <i>You do not need to edit this code.</i>

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this homework, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

We provide you with preprocessing code here, and you should not modify it.

In [ ]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [ ]:
### DO NOT EDIT THIS CELL ###

import torchtext
import random
def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

train_dataset, test_dataset = getDataset()

wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 97.3MB/s]


Run the next cell to see 10 random sentences of the training data.

In [ ]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'The', '<UNK>', 'Assembly', 'of', 'India', 'met', 'for', 'its', 'fifth', 'session', 'at', '11', 'pm', 'on', '14', 'August', 'in', 'the', 'Constitution', 'Hall', 'in', 'New', 'Delhi', '.', '</s>']
['<s>', 'It', 'is', 'available', 'to', 'purchase', 'as', 'a', 'CD', 'single', 'while', 'the', 'remixes', 'are', 'available', 'on', 'vinyl', '.', '</s>']
['<s>', 'With', 'the', 'trade', 'deadline', 'approaching', ',', 'speculation', 'picked', 'up', 'on', 'the', 'Blue', 'Jackets', 'trading', 'Carter', '.', '</s>']
['<s>', 'The', 'portion', 'of', 'heavier', 'elements', 'may', 'be', 'an', 'indicator', 'of', 'the', 'likelihood', 'that', 'the', 'star', 'has', 'a', 'planetary', 'system', '.', '</s>']
['<s>', 'At', 'the', 'dance', ',', 'a', 'senior', 'citizen', 'approaches', 'Ron', 'Swanson', 'and', 'asks', 'for', 'an', '<UNK>', 'from', 'Duke', 'Silver', '.', '</s>']
['<s>', 'Alternatively', ',', 'the', 'family', 'could', 'have', 'belonged', 'to', 'the', 'tribal', 'leadership', 'who', 'amassed

## The LanguageModel Class

You will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 15 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: <b>[5 points]</b> Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: <b>[5 points]</b> Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: <b>[5 points]</b> You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows: 

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

Feel free to implement helper methods as you wish (either in the base class or in the subclases). <b>But be sure not to change the function signatures of the provided methods</b> (i.e. the function and argument names), or else the autograder will fail.

In [ ]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided. 
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")
        
    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## <font color='red'>TODO:</font> Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

In [ ]:
class UnigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        self.unigramL = UnigramHelper(trainCorpus)

    def generateSentence(self):
        
        ### TODO ###
        sentenceEnd = False
        sentence = [START]
        while(not sentenceEnd):
            word = self.unigramL.generateWord()
            sentence.append(word)
            if word == END:
                sentenceEnd = True
        print(sentence)
        return sentence

    def getSentenceLogProbability(self, sentence):
        
        logprob = 0.0
        #sentence = sentence.split()
        for i in range(1,len(sentence)):
            logprob += self.unigramL.logProbability(sentence[i])     
        return logprob
        
    def getCorpusPerplexity(self, testCorpus):
        
        ### TODO ###
        N = 0.0
        perplex = 0.0
        for sen in testCorpus:
            for word in sen:
                if word != START:
                  perplex += self.unigramL.logProbability(word)
                  N += 1
        perplex =  -perplex / N
        perplex = math.exp(perplex)
        return perplex

class UnigramHelper:
    def __init__(self, trainCorpus):
        self.totalWordCount = 0.0
        self.wordNCount = defaultdict(float)
        self.training(trainCorpus)

    # Add observed wordNCount from trainCorpus to the distribution
    #This pretty much finds the count of each word and totalWordCount word of entire thing
    def training(self, trainCorpus):
        for sen in trainCorpus:
            for word in sen:
                if word != START:
                  self.totalWordCount += 1.0
                  self.wordNCount[word] += 1.0
            
    # Returns the probability of word in the distribution
    def logProbability(self, word):
        return math.log(self.wordNCount[word]/self.totalWordCount)

    # Generate a single random word according to the distribution
    def generateWord(self):
        rand = self.totalWordCount*random.random()
        for word in self.wordNCount:
            #print(word)
            rand -= self.wordNCount[word]
            if rand <= 0.0:
                return word
# End sample unigram dist code

We provide you with a testing function that uses very simple training & test corpora (you could compute probability/perplexity by hand if you wanted to). Note that passing this test does not guarantee you a perfect score in the autograder; this is simply to help you debug your model.

In [ ]:
def testModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of <unk> ( <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic parts of the game allowed for <unk> incredible city .", 
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]
    print(train_corpus)
    print(test_corpus)
    print(sentence)
    # These are the correct answers (don't change them!)
    if model_type == "unigram":
       senprobs = [-18.9159206916, -106.714608418, -107.8132207067, -43.0623556464, -54.9560026056]
       trainPerp, testPerp = 40.3970060507, 37.7244929883
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-18.8969788221, -107.3856946234, -108.078841804, -43.7800012747, -55.3816031464]
       trainPerp, testPerp = 41.0547195671, 39.356140682
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.4018400696, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.1336514995, -84.9068097328, -84.6431458887, -39.3603940053, -51.4605809045]
       trainPerp, testPerp = 18.6021115212, 28.8970586024
       model = SmoothedBigramModelAD(train_corpus)       

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        print(sen)
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    testModel('unigram')

[['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', '<UNK>', '(', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>'], ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']]
[['<s>', 'Classic', 'parts', 'of', 'the', 'game', 'allowed', 'for', '<UNK>', 'incredible', 'city', '.', '</s>'], ['<s>', 'Few', '<UNK>', 'realize', 'the', 'difficult', 'network', ',', 'which', 'linked', 'the', 'game', 'to', 'Sonic', '.', '</s>']]
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
--- TEST: generateSentence() ---
['<s>', 'to', ',', 'difficult', '.', 'rate', 'for', 'to', 'graphics', '.', '(', 'of', 'for', '</s>']
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) --

Now, you can train your model on the full WikiText corpus, and evaluate it on the held-out test set.

In [ ]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
['<s>', 'the', 'September', 'arrival', '10', 'also', '@-@', ',', 'the', 'use', 'a', 'daughter', 'that', 'the', 'in', '28', 'a', 'with', 'back', 'the', 'grey', 'as', 'brownish', 'Flocks', 'country', 'tubes', 'historic', 'the', 'by', 'during', 'Olympic', 'the', ',', 'from', 'foremast', '@-@', 'of', 'Aniston', 'poem', 'town', 'third', '<UNK>', 'a', 'Yeovil', '(', 'these', 'the', 'team', 'up', 'promoted', ',', 'linked', 'devotion', 'there', 'Final', '</s>']
Log Probability: -374.8175676313683 	Sentence: ['<s>', 'the', 'September', 'arrival', '10', 'also', '@-@', ',', 'the', 'use', 'a', 'daughter', 'that', 'the', 'in', '28', 'a', 'with', 'back', 'the', 'grey', 'as', 'brownish', 'Flocks', 'country', 'tubes', 'historic', 'the', 'by', 'during', 'Olympic', 'the', ',', 'from', 'foremast', '@-@', 'of', 'Aniston', 'poem', 'town', 'third', '<UNK>', 'a', 'Yeovil', '(', 'these', 'the', 'team', 'up', 'promoted', ',', 'linked', 'devotion', 'there', 'Final

## <font color='red'>TODO:</font> Smoothed Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth your model, you will need the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

In [ ]:
class SmoothedUnigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        self.smoothUnigram = SmoothUnigramHelper(trainCorpus)
        

    def generateSentence(self):

        ### TODO ###
        end = False
        sentence = [START]
        while(not end):
            word = self.smoothUnigram.generateWord()
            sentence.append(word)
            if word == END:
                end = True
        print(sentence)
        return sentence

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        logprob = 0.0
        for i in range(1,len(sentence)):
            logprob += self.smoothUnigram.logProbability(sentence[i])     
        return logprob
        
    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        N = 0.0
        perplex = 0.0
        for sen in testCorpus:
            for word in sen:
                if word != START:
                  perplex += self.smoothUnigram.logProbability(word)
                  N += 1
        perplex =  -perplex / N
        perplex = math.exp(perplex)
        return perplex

class SmoothUnigramHelper:
    def __init__(self, trainCorpus):
        self.totalWordCount = 0.0
        self.wordNCount = defaultdict(float)
        self.training(trainCorpus)

    # Add observed wordNCount from trainCorpus to the distribution
    #This pretty much finds the count of each word and totalWordCount word of entire thing
    def training(self, trainCorpus):
        for sen in trainCorpus:
            for word in sen:
                if word != START:
                  self.totalWordCount += 1.0
                  self.wordNCount[word] += 1.0
            
    # Returns the probability of word in the distribution
    def logProbability(self, word):
        # print(len(self.wordNCount))
        # print(self.wordNCount[word])
        # print(self.totalWordCount)
        # return math.log(self.wordNCount[word]/self.totalWordCount)
        return math.log((self.wordNCount[word]+1)/(len(self.wordNCount)+self.totalWordCount))
        # return math.log((self.wordNCount[word]/self.totalWordCount+1)/(len(self.wordNCount)+self.totalWordCount))

    # Generate a single random word according to the distribution
    def generateWord(self):
        rand = self.totalWordCount*random.random()
        for word in self.wordNCount:
            #print(word)
            rand -= self.wordNCount[word]
            if rand <= 0.0:
                return word
# End sample unigram dist code

In [ ]:
if __name__=='__main__':
    testModel('smoothed-unigram')

[['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', '<UNK>', '(', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>'], ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']]
[['<s>', 'Classic', 'parts', 'of', 'the', 'game', 'allowed', 'for', '<UNK>', 'incredible', 'city', '.', '</s>'], ['<s>', 'Few', '<UNK>', 'realize', 'the', 'difficult', 'network', ',', 'which', 'linked', 'the', 'game', 'to', 'Sonic', '.', '</s>']]
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
--- TEST: generateSentence() ---
['<s>', 'for', 'it', '</s>']
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -

In [ ]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
['<s>', '.', 'be', 'Road', 'height', '"', 'peak', '.', 'was', 'PopMart', 'night', 'that', 'indicates', '1991', 'sources', 'Villa', 'reported', '@,@', '200', 'the', '<UNK>', 'of', 'the', '.', 'politicians', ',', 'to', '.', '.', 'and', 'Steiner', 'and', ',', 'twelve', 'and', 'made', '@-@', '</s>']
Log Probability: -233.13118525635286 	Sentence: ['<s>', '.', 'be', 'Road', 'height', '"', 'peak', '.', 'was', 'PopMart', 'night', 'that', 'indicates', '1991', 'sources', 'Villa', 'reported', '@,@', '200', 'the', '<UNK>', 'of', 'the', '.', 'politicians', ',', 'to', '.', '.', 'and', 'Steiner', 'and', ',', 'twelve', 'and', 'made', '@-@', '</s>']
['<s>', 'believed', 'that', 'caused', ':', 'lyrics', 'of', 'recover', 'Vermeer', 'one', '</s>']
Log Probability: -74.4934341688063 	Sentence: ['<s>', 'believed', 'that', 'caused', ':', 'lyrics', 'of', 'recover', 'Vermeer', 'one', '</s>']
['<s>', 'Napa', 'allowing', 'majority', 'bit', 'note', '.', 'their', 'tw

## <font color='red'>TODO:</font> Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

In [ ]:
class BigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        self.bigramL = BigramL(trainCorpus)

    def generateSentence(self):
        # return []
        ### TODO ###
        end = False
        sentence = [START]
        prevWord = START
        counter = 0
        while(not end):
          #prevWord starts STARTS and now word = By, Few
          word = self.bigramL.generateWord(prevWord)
          sentence.append(word)
          prevWord = word # prevWord = By , Few
          if word == END:
            end = True
        # sentence.append(END)
        # print(sentence)
        return sentence

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        probability = 0.0
        for i in range(1,len(sentence)):
          probability += self.bigramL.prob(sentence[i],sentence[i-1])
        return probability
        
    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        N = 0.0
        perplex = 0.0
        for i in range(len(testCorpus)):
            for j in range(1,len(testCorpus[i])):
                # if word != START:
                perplex += self.bigramL.prob(testCorpus[i][j],testCorpus[i][j-1])
                N += 1
        perplex =  -perplex / N
        perplex = math.exp(perplex)
        return perplex

class BigramL:
    def __init__(self, trainCorpus):
      self.unicounts = defaultdict(float)
      self.bicounts = defaultdict(float)
      self.train(trainCorpus)

    def train(self, trainCorpus):
      for i in range(len(trainCorpus)):
        # print(trainCorpus[i])
        for j in range(len(trainCorpus[i])):
          if j != 0:
            # print((trainCorpus[i][j],trainCorpus[i][j-1]))
            self.bicounts[(trainCorpus[i][j],trainCorpus[i][j-1])] += 1.0
          self.unicounts[trainCorpus[i][j]] += 1.0
          # print(self.bicounts)
          # print(self.unicounts)
    
    def prob (self, word, prevWord):
      valueProb = self.bicounts[word, prevWord]/self.unicounts[prevWord]
      if valueProb == 0:
        return float('-inf')
      return math.log(valueProb)

    def generateWord(self, prevWord):
      rand = self.unicounts[prevWord]*random.random() #it is receiving previous word
      # print(prevWord)
      for w1, w2 in self.bicounts:
        # print((w1,w2))
        if w2 == prevWord:
          rand -= self.bicounts[w1,w2]
          if rand <= 0.0:
            #print(w1) #so it is printing the next word By, Few
            return w1



In [ ]:
if __name__=='__main__':
    testModel('bigram')

[['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', '<UNK>', '(', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>'], ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']]
[['<s>', 'Classic', 'parts', 'of', 'the', 'game', 'allowed', 'for', '<UNK>', 'incredible', 'city', '.', '</s>'], ['<s>', 'Few', '<UNK>', 'realize', 'the', 'difficult', 'network', ',', 'which', 'linked', 'the', 'game', 'to', 'Sonic', '.', '</s>']]
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 P

In [ ]:
if __name__=='__main__':
    runModel('bigram')

--------- 5 sentences from your model ---------
Log Probability: -42.777810429339084 	Sentence: ['<s>', 'Tech', 'Revolution', ',', 'the', 'film', "'s", 'jurisdiction', 'over', 'Stan', 'Wawrinka', '.', '</s>']
Log Probability: -79.17259031754047 	Sentence: ['<s>', 'The', 'new', '<UNK>', 'in', 'the', 'songs', ',', 'an', 'unpleasant', 'thing', 'I', 'am', 'after', 'admitting', '[', 'was', 'an', 'arena', '.', '</s>']
Log Probability: -39.3562002297394 	Sentence: ['<s>', 'The', 'view', 'was', 'at', '2', 'June', '1937', '.', '</s>']
Log Probability: -98.28867860516424 	Sentence: ['<s>', 'The', 'following', 'their', 'homes', 'in', '1985', ',', '5th', 'Division', 'on', 'certain', 'is', 'not', 'research', ',', 'and', '<UNK>', '<UNK>', 'National', 'Union', 'for', '"', '.', '</s>']
Log Probability: -55.697451948023314 	Sentence: ['<s>', 'In', 'addition', ',', 'Fernandez', 'was', 'the', 'final', ',', 'Gabrielle', 'Solis', '(', 'III', ')', '.', '</s>']

--------- Corpus Perplexities ---------
Traini

## <font color='red'>TODO:</font> Smoothed Bigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth your model, you need to compute a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, you can compute $D$ as: 

$$D=\frac{n_1}{n_1+2n_2}$$ 

For each word $w$, you then need to compute the number of bigram types $ww’$ as follows: 

$$S(w)=|\{w’|c(ww’)>0\}|$$ 

where $c(ww’)$ is the frequency of $ww’$ in the training data.

Finally, you can compute $P_{AD}(w’|w)$ as follows: 

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$ 

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [ ]:
class SmoothedBigramModelAD(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        self.smoothBigram = SmoothBigramHelper(trainCorpus)

    def generateSentence(self):

        ### TODO ###
        sentence = [START]
        end = False
        prev_word = START
        while(not end):
          word = self.smoothBigram.generate(prev_word)
          sentence.append(word)
          prev_word = word
          if word == END:
            end = True
        return sentence

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        #pass
        probability = 0.0
        for i in range(1,len(sentence)):
          probability +=self.smoothBigram.prob(sentence[i],sentence[i-1])
        return probability
        
    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        perplex = 0.0
        N = 0.0
        for i in range(len(testCorpus)):
          for j in range(1,len(testCorpus[i])):
            perplex += self.smoothBigram.prob(testCorpus[i][j],testCorpus[i][j-1])
            N += 1
        perplex = -perplex / N
        perplex = math.exp(perplex)
        return perplex


class SmoothBigramHelper:
    def __init__(self, trainCorpus):
      self.bicounts = defaultdict(float)
      self.unicounts = defaultdict(float)
      self.smoothUnigram = SmoothUnigramHelper(trainCorpus)
      self.total = 0.0
      self.D = 0.0
      self.train(trainCorpus)
      self.computeD()
      self.Sw = defaultdict(float)
      self.computeSw()
    #pass
    def train(self, trainCorpus):
      for i in range(len(trainCorpus)):
        for j in range(len(trainCorpus[i])):
          self.unicounts[trainCorpus[i][j]] += 1.0
          if j !=0:
            self.bicounts[trainCorpus[i][j],trainCorpus[i][j-1]] += 1.0
          self.total += 1.0

    def computeD(self):
      n1 = sum(x == 1 for x in self.bicounts.values())
      n2 = sum(x == 2 for x in self.bicounts.values())
      self.D = float(n1) / (n1 + 2*n2)
    #pass
    def computeSw(self):
      for (w1, w2) in self.bicounts:
        self.Sw[w2] += 1
    #pass
    def prob(self, word, prev_word):
      prob = max((self.bicounts[word,prev_word] - self.D), 0)/self.unicounts[prev_word]
      temp = math.exp(self.smoothUnigram.logProbability(word))
      prob += self.D * self.Sw[prev_word] * temp / self.unicounts[prev_word]
      return math.log(prob)
    #pass
    def generate(self, prev_word):
      rand = self.unicounts[prev_word]*random.random()
      for word in self.unicounts:
        if word != START:
          rand -= max((self.bicounts[word,prev_word] -self.D),0) + self.D * self.Sw[prev_word] * math.exp(self.smoothUnigram.logProbability(word))
        if rand <= 0.0:
          return word


In [ ]:
if __name__=='__main__':
    testModel('smoothed-bigram')

[['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', '<UNK>', '(', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>'], ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']]
[['<s>', 'Classic', 'parts', 'of', 'the', 'game', 'allowed', 'for', '<UNK>', 'incredible', 'city', '.', '</s>'], ['<s>', 'Few', '<UNK>', 'realize', 'the', 'difficult', 'network', ',', 'which', 'linked', 'the', 'game', 'to', 'Sonic', '.', '</s>']]
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -16.1336514995 	Your log prob.

In [ ]:
if __name__=='__main__':
    runModel('smoothed-bigram')

--------- 5 sentences from your model ---------
Log Probability: -70.93446739549101 	Sentence: ['<s>', 'Meddings', "'", 'their', 'season', 'they', 'do', 'the', 'building', 'while', 'a', 'black', 'men', '.', '</s>']
Log Probability: -32.96108723986717 	Sentence: ['<s>', 'After', 'km2', ')', 'is', 'named', 'the', '</s>']
Log Probability: -12.29909898698821 	Sentence: ['<s>', 'During', 'the', '"', '.', '</s>']
Log Probability: -227.72728305777022 	Sentence: ['<s>', '"', 'they', 'were', 'planted', 'along', 'with', 'whom', 'he', 'wrote', 'that', 'the', 'verge', 'of', 'the', 'destruction', 'of', '2013', ',', 'asphalt', 'and', 'the', 'summer', 'of', 'M', '@-@', 'drama', ',', 'and', 'with', 'no', 'that', 'the', 'story', 'of', 'praise', 'campaign', 'for', 'the', 'highest', 'civilian', 'life', 'at', 'a', 'Great', 'troops', 'into', 'the', 'famous', '</s>']
Log Probability: -88.83411360567041 	Sentence: ['<s>', 'Thus', ',', '"', 'True', "'s", 'first', 'series', 'has', 'four', 'years', ',', 'Mr', '

## Food for Thought
We provide you some questions to think about. <b>You do not need to answer these questions, but we encourage you to give them some thought.</b>
<ol>
<li>When generating sentences with the unigram model, what controls the length of the generated sentences? How does this differ from the sentences produced by the bigram models?
<li>Consider the probability of the generated sentences according to your models. Do your models assign drastically different probabilities to the different sets of sentences? Why do you think that is?
<li>Look back at the sentences generated using your models. In your opinion, which model produces better / more realistic sentences?
<li>For each of the four models, which test corpus has the highest perplexity? Why?
<li> Why do you think it might be a bad idea to use Laplace (add-one) smoothing for a bigram model? How does the absolute discounting method help?
</ol>

# Part 2: Finite State Transducers [40 points]

Here, you will implement a <b>finite state transducer</b> (FST), which transduces the infinitive form of verbs to the present participle (<em>-ing</em>) form. You will be graded according to how well your transducer performs on a hidden test dataset.

Run the cell below to pull the data you can use to develop your FST.

## Provided Code
We provide you with a class <TT>FST</TT>, which is a module for constructing and evaluating FST's. You shouldn't modify any of this code. Here is a description of the most useful methods of this module:


* <b>`FST(self, initialStateName)`</b>: Instantiate an FST with an initial (non-accepting) state named `initialStateName`

* <b>`addState(self, name, isFinal=False)`</b>: Add a state `name` to the FST; by default, `isFinal=False` and so the state is not an accepting state.

* <b>`addTransition(self, inStateName, inString, outString, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName`, where both of these states already exist in the FST. The FST can traverse this transition after reading `inString`, and outputs `outString` when it does so.

* <b>`addSetTransition(self, inStateName, inStringSet, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName` for each character in `inStringSet`. For each transition, the FST outputs the same character it reads.

The cell also contains a method to read the data and a scoring function that you will call in the "Test your FST" section.

<b>Do not edit any of the code in this cell!</b>


In [1]:
### DO NOT EDIT THIS CELL ###

import os
import pandas as pd
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql', showindex=False)

def readVerbFile(file):
    url='https://drive.google.com/uc?id=1_BKoF40-TaqKvQy3H05GwP1w_VkpZDsh'
    df = pd.read_csv(url, header=None)

    return df.values.tolist()

verbs = readVerbFile('verbsList.csv')

def testFST(print_examples = 'all'):
    assert print_examples in {'all', 'incorrect', 'none'}, "print_examples must be 'all', 'incorrect', or 'none'"
    rule_classes = [('0', "0 Rule: Add -ing", 'walk    ==>  walking'), ('1-cons', "1 Rule: Drop final -e after consonant", 'ride    ==>  riding'), ('1-u', "1 Rule: Drop final -e after u", 'argue   ==>  arguing'), ('1*', "1 Exception: Don't drop -e", 'see     ==>  seeing'), ('2-b', "2 Rule: Double -b", 'grab    ==>  grabbing'), ('2-d', "2 Rule: Double -d", 'nod     ==>  nodding'), ('2-g', "2 Rule: Double -g", 'tag     ==>  tagging'), ('2-m', "2 Rule: Double -m", 'slam    ==>  slamming'), ('2-n', "2 Rule: Double -n", 'ban     ==>  banning'), ('2-p', "2 Rule: Double -p", 'stop    ==>  stopping'), ('2-r', "2 Rule: Double -r", 'bar     ==>  barring'), ('2-t', "2 Rule: Double -t", 'set     ==>  setting'), ('2*-b', "2 Exception: Don't double -b", 'climb   ==>  climbing'), ('2*-d', "2 Exception: Don't double -d", 'lead    ==>  leading'), ('2*-g', "2 Exception: Don't double -g", 'swing   ==>  swinging'), ('2*-m', "2 Exception: Don't double -m", 'harm    ==>  harming'), ('2*-n', "2 Exception: Don't double -n", 'clean   ==>  cleaning'), ('2*-p', "2 Exception: Don't double -p", 'leap    ==>  leaping'), ('2*-r', "2 Exception: Don't double -r", 'roar    ==>  roaring'), ('2*-t', "2 Exception: Don't double -t", 'halt    ==>  halting'), ('2*-er', "2 Exception: Verbs ending in -er", 'gather  ==>  gathering'), ('2*-en', "2 Exception: Verbs ending in -en", 'happen  ==>  happening'), ('3', "3 Rule: Change -ie to -y", 'die     ==>  dying')]

    f = buildFST()
    #verbs = [v for v in verbs if v[0] in {'canoe', 'hoe', 'shoe', 'snowshoe', 'tiptoe', 'toe'}]
    myParses = f.parseInputList([x[0] for x in verbs])
    scores, totals, examples = {}, {}, []

    for i in range(len(verbs)):
        lemma, form, clas = verbs[i]
        output = myParses[i]
        scores[clas] = scores.get(clas, 0)
        totals[clas] = totals.get(clas, 0) + 1
        if print_examples == 'all' or print_examples == 'incorrect' and form != output:
            examples += [(lemma, form, output, 'CORRECT' if form == output else 'INCORRECT')]
        if form == output: scores[clas] += 1
    
    if print_examples != 'none' and len(examples) > 0:
        examples = pd.DataFrame.from_records(examples, columns = ['Input', 'Correct Output', 'Returned Output', 'Result'])
        print(pdtabulate(examples))

    data= []
    
    # We use a scoring method that accounts for (1) the fact that the default FST gets many verbs correct
    #                                           (2) the class inbalance among the different rules
    # p_scores is for verbs that the default FST gets correct, q_scores for verbs it gets wrong
    # Each contains a list of accuracies for each category group (for example, all of the examples where rule 2 applies are in one group)
    p_scores, q_scores = {'0': [], '1*': [], '2*': [], '2*-en/er': []}, {'1': [], '2': [], '3': []}
    for clas, msg, ex in rule_classes:
        acc = scores[clas]/totals[clas]
        data += [(msg, ex, scores[clas], totals[clas], 100*acc)]
        if clas == '0': p_scores['0'].append(acc)
        elif clas in {'1-cons', '1-u'}: q_scores['1'].append(acc)
        elif clas == '1*': p_scores['1*'].append(acc)
        elif '2' in clas and '*' not in clas: q_scores['2'].append(acc)
        elif '2*' in clas:
            if 'er' in clas or 'en' in clas: p_scores['2*-en/er'].append(acc)
            else: p_scores['2*'].append(acc)
        elif '3' in clas: q_scores['3'].append(acc)
        else: assert False, 'should not get here'

    p_scores = [sum(v) / len(v) for k, v in p_scores.items()] # Get the average for each category group
    q_scores = [sum(v) / len(v) for k, v in q_scores.items()]

    # Score by averaging the p and q scores, then applying grading formula
    p, q = sum(p_scores) / len(p_scores), sum(q_scores) / len(q_scores)
    final_score = q/2 * (1+p) # Score will be 0 when p is 100% and q is 0%; score will be 100% when p and q are both 100%

    print('\nScorecard:')
    data = pd.DataFrame.from_records(data, columns = ['Category', 'Example', 'Correct', 'Total', 'Accuracy (%)'])
    print(pdtabulate(data))

    print("Overall Score:", str(final_score*100) + '%')

class Transition:
    # string_in
    # string_out
    def __init__(self, inState, inString, outString, outState):
        self.state_in = inState
        self.string_in = inString
        self.string_out = outString
        self.state_out = outState

    def equals(self, t):
        if self.state_in == t.state_in \
        and self.string_in == t.string_in \
        and self.string_out == t.string_out \
        and self.state_out == t.state_out:
            return True
        else:
            return False

class FSTstate:
    # id: an integer ID of the state
    # isFinal: is this a final state?
    def __init__(self, n, isF, fst):
        self.id = n
        self.isFinal = isF
        self.transitions = dict() # map inStrings to a set of all possible transitions
        self.FST = fst

    def addTransition(self, inString, outString, outState):
        newTransition = Transition(self, inString, outString, outState)
        if inString in self.transitions:
            for t in self.transitions[inString]:
                if t.equals(newTransition):
                    return
            self.transitions[inString].add(newTransition)
        else:
            self.transitions[inString] = set([])
            self.transitions[inString].add(newTransition)
    
    def parseInputFromStartState(self, inString):
        parseTuple = ("", self.id)
        parses = []
        (accept, stringParses) = self.parseInput(inString)
        if accept:
            for p in stringParses:
                completeParse = [parseTuple]
                completeParse.extend(p)
                parses.append(completeParse)
        return (accept, parses)

    def parseInput(self, inString):
        parses = []
        isAccepted = True
        
        DEBUG = False
        if DEBUG:
            print("parseInput: state: ", self.id, " parsing: " , inString)
        
        # Case 1: no suffix
        if inString == "":
            epsilonParses = []
            epsilonAccepted = False
            # try all epsilon transitions
            if "" in self.transitions:
                transSet = self.transitions[""]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(inString)
                    if suffixAccepted:
                        epsilonAccepted = True
                        if suffixParses == []: #accepts.
                            parse_s = [parseTuple]
                            epsilonParses.append(parse_s)
                        else:
                            for s in suffixParses:
                                parse_s = [parseTuple]
                                parse_s.extend(s)
                                epsilonParses.append(parse_s)
            # if epsilon is accepted, add all its parses
            if epsilonAccepted:
                parses.extend(epsilonParses)
            # if this is a final state, add an empty parse
            if self.isFinal or parses != []:
                if DEBUG:
                    print("Accepted in state ", self.id)
                return (True, parses)
            else:
                if DEBUG:
                    print("Rejected in state ", self.id)
                return (False, None)
        # case 2: non-empty suffix: there needs to be one suffix that parses!)
        hasAcceptedSuffix = False;
        for i in range(0,len(inString)+1):
            prefix = inString[0:i]
            suffix = inString[i:len(inString)]
            if DEBUG:
                print("\t prefix: \'", prefix, "\' I=", i)
            if prefix in self.transitions:
                if DEBUG:
                     print("\t prefix: ", prefix,  "suffix: ", suffix, "I=", i)
                transSet = self.transitions[prefix]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(suffix)
                    if suffixAccepted:
                        hasAcceptedSuffix = True
                        if suffixParses == []:
                            parse_s = [parseTuple]
                            parses.append(parse_s)
                            thisPrefixParses = True
                        for s in suffixParses:
                            parse_s = [parseTuple]
                            parse_s.extend(s)
                            parses.append(parse_s)
        if hasAcceptedSuffix:
            return (True, parses)
        else:
            return (False, None)
                            


    def printState(self):
        if self.isFinal:
            FINAL = "FINAL"
        else: FINAL = ""
        print("State", self.id, FINAL)
        for inString in self.transitions:
            transList = self.transitions[inString]
            for t in transList:
                print("\t", inString, ":", t.string_out, " => ", t.state_out)

                    


class FST:
    def __init__(self, initialStateName="q0"):
        self.nStates = 0
        self.initState = FSTstate(initialStateName, False, self) 
        self.allStates = dict()
        self.allStates[initialStateName] = self.initState
       
    def addState(self, name, isFinal=False):
        if name in self.allStates:
            print("ERROR addState: state", name, "exists already")
            sys.exit()
        elif len(self.allStates) >= 30:
            print("ERROR addState: you can't have more than 30 states")
            sys.exit()
        else:  
            newState = FSTstate(name, isFinal, self)
            self.allStates[name] = newState

    def addTransition(self, inStateName, inString, outString, outStateName):
        if (len(inString) > 1):
            print("ERROR: addTransition: input string ", inString, " is longer than one character")
            sys.exit()
        if inStateName not in self.allStates:
            print("ERROR: addTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addTransition: state ", outStateName, " does not exist")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition(inString, outString, outStateName)

    # epsilon:epsilon
    def addEpsilonTransition(self, inStateName, outStateName):
        if inStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", outStateName, " does not exist")
            sys.exit()
        if inStateName == outStateName:
            print("ERROR: we don't allow epsilon loops")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition("", "", outStateName)

    # map every element in inStringSet to itself
    def addSetTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, s, outStateName)

    # map string to itself
    def addSelfTransition(self, inStateName, inString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         self.addTransition(inStateName, inString, inString, outStateName)

    # map every element in inStringSet to outString
    def addSetToStringTransition(self, inStateName, inStringSet, outString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, outString, outStateName)
    
            
    # map every element in inStirngSet to outString
    def addSetEpsilonTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetEpsionTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, "", outStateName)
            
    def parseInput(self, inString):
        SHOW_STATES = False#True
        inString = inString.rstrip('\n')
        (canParse, allParses)  = self.initState.parseInputFromStartState(inString)
        allParsesAsString = ""
        if canParse:
            for parse in allParses:
                for tuple in parse:
                    outString, outState = tuple
                    allParsesAsString += outString
                if SHOW_STATES:
                    allParsesAsString += "\t  States: "
                    i = 0
                    for tuple in parse:
                        i += 1
                        outString, outState = tuple
                        allParsesAsString += outState
                        if i < len(parse):
                            allParsesAsString += " => "
                    allParsesAsString += "; "
          
            return True, allParsesAsString
        else:
            return False, "FAIL"

    def printFST(self):
        print("Printing FST", str(self))
        for stateID in self.allStates:
            state = self.allStates[stateID]
            state.printState()

    def parseInputList(self, verbList):
        #with open(fileName, "r") as f:
        nParses = 0
        totalStrings = 0
        res = []
        for verb in verbList:#f:
            totalStrings += 1
            canParse, parse = self.parseInput(verb)
            res += [parse]
            if canParse:
                nParses += 1
        fraction = nParses/totalStrings
        print(nParses, "/", totalStrings, "=", str(fraction*100)+'%', "of examples parsed") 
        return res

## Present Participle Transduction
Here we provide you with a specification for producing the <em>-ing</em> form for English verbs. You may assume that all verb stems are at least three letters long.

In many cases, the <em>-ing</em> form just consists of <TT>stem + ing</TT>:

<ul>
<TT>walk ==> walking&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fly ==> flying</TT>
</ul>

But there are a number of exceptions. In particular, you will need to implement the following rules:
<ol>
<li><b>Drop the final <em>-e</em>:</b> Drop the final <em>-e</em> <b>if and only if</b> it is preceded by a consonant or by the letter <em>u</em>:
<ul>
<em>Examples</em>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<em>Exceptions</em><br>
<TT>ride ==> riding&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;see ==> seeing</TT><br>
<TT>argue ==> arguing&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;canoe ==> canoeing</TT>
</ul>
<li><b>Double the final consonant:</b> Double a final single <em>-b</em>, <em>-d</em>, <em>-g</em>, <em>-m</em>, <em>-n</em>, <em>-p</em>, <em>-r</em>, <em>-t</em> <b>if and only if</b> it is preceded by a single vowel:
<ul>
<em>Examples</em>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<em>Exceptions</em><br>
<TT>grab ==> grabbing&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;climb ==> climbing</TT><br>
<TT>nod ==> nodding&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;lead ==> leading</TT><br>
<TT>tag ==> tagging&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;swing ==> swinging</TT><br>
<TT>slam ==> slamming&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;harm ==> harming</TT><br>
<TT>ban ==> banning&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;clean ==> cleaning</TT><br>
<TT>stop ==> stopping&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;leap ==> leaping</TT><br>
<TT>bar ==> barring&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;roar ==> roaring</TT><br>
<TT>set ==> setting&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;halt ==> halting</TT><br>
</ul>
<b>Exceptions</b> to this rule are verbs ending in <em>-er</em> or <em>-en</em>:
<ul>
<TT>gather ==> gathering&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;happen ==> happening&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</TT>
</ul>
We are assuming American English spelling for words like <em>labeling</em>. Note also that this is a simplification of the actual spelling rule; our data does not contain verbs like <em>pardon</em>, where this rule does not apply, or verbs like <em>deter</em>, where this rule does apply.
<li> <b>Change the final <em>-ie</em> to <em>-y</em>:</b>
<ul>
<TT>die ==> dying</TT>
</ul>
</ol>


## <font color='red'>TODO:</font> Build Your FST [40 points]

We have provided you with a rudimentary (and buggy) implementation for transducing verbs to the <em>-ing</em> form. Your task is to modify the function <TT>buildFST()</TT> below to produce a better FST. You are free to define your own character sets or other helper methods that you require. 

The FST will finish its analysis for a string after having read its last character. If it accepts the string, it will output the transduced form; if it does not accept it, the FST will output FAIL. Your FST may be non-deterministic (that is, a state may have multiple transitions for the same input character), but each accepted string should only have one analysis. If your FST accepts a string along multiple paths, all of the outputs will be returned and the translation will be marked incorrect: e.g. <TT>appeal ==> appealingappealling</TT>. It will accept a string if it reaches a final state after having read its last character.

In [3]:
# Here are some predefined character sets that might come in handy. Feel free to make your own.
AZ = set("abcdefghijklmnopqrstuvwxyz")
VOWS = set("aeiou")
CONS = AZ-VOWS
E = set('E')
AO = set("ao")
BDGMNPRT = set("bdgmnprt")
BNPRT = set("bnprt")
PT = set("pt")
E = set("e")
I = set("i")
U = set("u")

### TODO: Impelement your solution here. ###
def buildFST():

    # The states (you need to add more)
    f = FST("q0") # q0 is the initial (non-accepting) state
    f.addState("q1") # a non-accepting state
    f.addState("q_ing") # a non-accepting state
    f.addState("q_EOW", True) # an accepting state (you shouldn't need any additional accepting states)
    f.addState("q2") # a non-accepting state
    f.addState("q3") # a non-accepting state
    f.addState("q4") # a non-accepting state
    f.addState("q5") # a non-accepting state
    f.addState("q6") # a non-accepting state
    f.addState("q7") # a non-accepting state
    f.addState("q8") # a non-accepting state
    f.addState("q9") # a non-accepting state
    f.addState("q10") # a non-accepting state
    f.addState("q11") # a non-accepting state
    f.addState("q12") # a non-accepting state
    f.addState("q13") # a non-accepting state N
    f.addState("q14") # a non-accepting state P
    f.addState("q15") # a non-accepting state T
    f.addState("q16") # a non-accepting state R
    f.addState("q17") # a non-accepting state B
    f.addState("q18") # a non-accepting state D
    f.addState("q19") # a non-accepting state G
    f.addState("q20") # a non-accepting state M

    # The transitions (you need to add more):
    # f.addSetTransition("q0", AZ, "q1") # transduce every element in this set to itself
    # f.addSetTransition("q1", AZ-E, "q1") # AZ-E =  the set AZ without the elements in the set E

    f.addSetTransition("q0", CONS, "q1")
    f.addSetTransition("q0", VOWS, "q2")
    # AZ-E =  the set AZ without the elements in the set E
    f.addSetTransition("q1", CONS, "q1")
    f.addSetTransition("q1", AO, "q2")
    f.addTransition("q1", "e", "", "q3")
    f.addTransition("q1", "i", "", "q4")
    f.addSetTransition("q1", U, "q5")
    f.addTransition("q1", "", "", "q_ing")

    f.addSetTransition("q2", CONS-BDGMNPRT, "q1")
    f.addTransition("q2", "n", "n", "q13")
    f.addTransition("q2", "p", "p", "q14")
    f.addTransition("q2", "t", "t", "q15")
    f.addTransition("q2", "r", "r", "q16")
    f.addTransition("q2", "b", "b", "q17")
    f.addTransition("q2", "d", "d", "q18")
    f.addTransition("q2", "g", "g", "q19")
    f.addTransition("q2", "m", "m", "q20")
    f.addSetTransition("q2", VOWS, "q10")
    f.addTransition("q2", "","", "q_ing")

    f.addTransition("q3", "", "e", "q6")
    f.addTransition("q3", "","", "q_ing")

    f.addTransition("q4", "", "i", "q7")
    f.addTransition("q4", "e", "", "q11")

    f.addTransition("q5", "e", "", "q8")
    f.addSetTransition("q5", CONS-BDGMNPRT, "q1")
    f.addSetTransition("q5", VOWS-E, "q10")
    f.addTransition("q5", "n", "n", "q13")
    f.addTransition("q5", "p", "p", "q14")
    f.addTransition("q5", "t", "t", "q15")
    f.addTransition("q5", "r", "r", "q16")
    f.addTransition("q5", "b", "b", "q17")
    f.addTransition("q5", "d", "d", "q18")
    f.addTransition("q5", "g", "g", "q19")
    f.addTransition("q5", "m", "m", "q20")
    f.addTransition("q5", "","", "q_ing")

    f.addSetTransition("q6", CONS-PT, "q1")
    f.addSetTransition("q6", VOWS, "q10")
    f.addTransition("q6", "p", "p", "q14")
    f.addTransition("q6", "t", "t", "q15")

    f.addSetTransition("q7", CONS-BDGMNPRT, "q1")
    f.addSetTransition("q7", VOWS-E, "q10")
    f.addTransition("q7", "n", "n", "q13")
    f.addTransition("q7", "p", "p", "q14")
    f.addTransition("q7", "t", "t", "q15")
    f.addTransition("q7", "r", "r", "q16")
    f.addTransition("q7", "b", "b", "q17")
    f.addTransition("q7", "d", "d", "q18")
    f.addTransition("q7", "g", "g", "q19")
    f.addTransition("q7", "m", "m", "q20")
    f.addTransition("q7", "","", "q_ing")

    f.addTransition("q8", "", "", "q_ing")
    f.addTransition("q8", "", "e", "q9")

    f.addSetTransition("q9", CONS, "q1")
    f.addSetTransition("q9", VOWS, "q10")

    f.addSetTransition("q10", CONS, "q1")
    f.addSetTransition("q10", VOWS, "q10")
    f.addTransition("q10", "", "", "q_ing")

    f.addTransition("q11", "", "y", "q_ing")
    f.addTransition("q11", "", "ie", "q12")

    f.addSetTransition("q12", VOWS, "q10")
    f.addSetTransition("q12", CONS, "q1")

    f.addTransition("q13", "", "n", "q_ing")
    f.addSetTransition("q13", CONS, "q1")
    f.addSetTransition("q13", AO, "q2")
    f.addTransition("q13", "e", "", "q3")
    f.addTransition("q13", "i", "", "q4")
    f.addTransition("q13", "u", "u", "q5")
    #m
    f.addTransition("q20", "", "m", "q_ing")
    f.addSetTransition("q20", CONS, "q1")
    f.addSetTransition("q20", AO, "q2")
    f.addTransition("q20", "e", "", "q3")
    f.addTransition("q20", "i", "", "q4")
    f.addTransition("q20", "u", "u", "q5")

    f.addTransition("q14", "", "p", "q_ing")
    f.addSetTransition("q14", CONS, "q1")
    f.addSetTransition("q14", AO, "q2")
    f.addTransition("q14", "e", "", "q3")
    f.addTransition("q14", "i", "", "q4")
    f.addTransition("q14", "u", "u", "q5")
    #d
    f.addTransition("q18", "", "d", "q_ing")
    f.addSetTransition("q18", CONS, "q1")
    f.addSetTransition("q18", AO, "q2")
    f.addTransition("q18", "e", "", "q3")
    f.addTransition("q18", "i", "", "q4")
    f.addTransition("q18", "u", "u", "q5")

    f.addTransition("q15", "", "t", "q_ing")
    f.addSetTransition("q15", CONS, "q1")
    f.addSetTransition("q15", AO, "q2")
    f.addTransition("q15", "e", "", "q3")
    f.addTransition("q15", "i", "", "q4")
    f.addTransition("q15", "u", "u", "q5")
    #g
    f.addTransition("q19", "", "g", "q_ing")
    f.addSetTransition("q19", CONS, "q1")
    f.addSetTransition("q19", AO, "q2")
    f.addTransition("q19", "e", "", "q3")
    f.addTransition("q19", "i", "", "q4")
    f.addTransition("q19", "u", "u", "q5")

    f.addTransition("q16", "", "r", "q_ing")
    f.addSetTransition("q16", CONS, "q1")
    f.addSetTransition("q16", AO, "q2")
    f.addTransition("q16", "e", "", "q3")
    f.addTransition("q16", "i", "", "q4")
    f.addTransition("q16", "u", "u", "q5")
    #b
    f.addTransition("q17", "", "b", "q_ing")
    f.addSetTransition("q17", CONS, "q1")
    f.addSetTransition("q17", AO, "q2")
    f.addTransition("q17", "e", "", "q3")
    f.addTransition("q17", "i", "", "q4")
    f.addTransition("q17", "u", "u", "q5")
    # Get rid of this transition! (it overgenerates)
    # f.addSetTransition("q1", AZ, "q_ing")

    # Map the empty string to ing: 
    f.addTransition("q_ing", "", "ing", "q_EOW")

    # Return your completed FST
    return f


## Test your FST
We have provided you with a dataset to help you debug and test your FST. Run the cell below to see what verbs your FST handles correctly and your overall score. To get a high score, you must perform well on each of the rules. Note that the autograder uses the same scoring function on a <b>different</b> dataset with the same type of verbs. If you are interested, you can find the scoring function in the "Provided Code" section above.

The `print_examples` argument of this method may be set to one of the following values:
* `all`: Print your FST's output on all of the words in the dataset.
* `incorrect`: Print your FST's output on only the words your FST gets incorrect.
* `none`: Don't print any of the words (only show the score).

In [ ]:
if __name__ == '__main__':
    testFST(print_examples='all')

521 / 521 = 100.0% of examples parsed
+--------------+------------------+-------------------+-----------+
| Input        | Correct Output   | Returned Output   | Result    |
|--------------+------------------+-------------------+-----------|
| employ       | employing        | employing         | CORRECT   |
| spy          | spying           | spying            | CORRECT   |
| worry        | worrying         | worrying          | CORRECT   |
| talk         | talking          | talking           | CORRECT   |
| fill         | filling          | filling           | CORRECT   |
| cancel       | canceling        | canceling         | CORRECT   |
| hijack       | hijacking        | hijacking         | CORRECT   |
| know         | knowing          | knowing           | CORRECT   |
| crash        | crashing         | crashing          | CORRECT   |
| block        | blocking         | blocking          | CORRECT   |
| wash         | washing          | washing           | CORRECT   |
| mark    

# What You Need to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then rename it to `hwk1.py` and submit it to the autograder in Gradescope.